In [ ]:
import xarray as xr
import xgcm
from eofs.xarray import Eof
import numpy as np
import xroms
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
from glob import glob
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

%matplotlib inline

In [ ]:
# Define map plotting function
'''xxx'''
def PlotMap(x1, x2, y1, y2):    
    coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
                            facecolor='none', name='coastline')
    roi = [y1, y2, x2, x1]
    ax2 = plt.axes(projection=ccrs.PlateCarree())
    feature = ax2.add_feature(coast, edgecolor='black',facecolor='gray')
    ax2.set_extent(roi)
    gl = ax2.gridlines(draw_labels=True,
             color='black', alpha=0.2, linestyle='--')
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.ylabels_right = False
    gl.xlabels_top = False

In [ ]:
files = glob('/srv/scratch/z3533156/20year_run_tide/outer_avg/outer_avg_*.nc')
# A basic chunk choice
chunks = {'ocean_time':1}

ds = xroms.open_mfnetcdf(files, chunks=chunks)

## calculate EOFs of SSH

In [ ]:
#ssh = xr.open_dataset('/srv/scratch/z3526974/EAC_22yr/dipole_work/EAC_roms_22yr_SSH.nc')

In [ ]:
ssh

In [ ]:
# # Set up solver (need to give it the right coordinates and variables to  match the dataset)
# coslat = np.cos(np.deg2rad(ds.coords['lat_rho'].values))
# wgts = np.sqrt(coslat)[..., np.newaxis]
# solver = Eof(ssh.zeta)#, weights=wgts)

In [ ]:
# # Retrieve the first 5 EOFs, expressed as the correlation between the leading
# # PC time series and the input sea level anomalies at each grid point, and the
# # leading PC time series itself.
# eof1 = solver.eofs(neofs=5)
# pc1 = solver.pcs(npcs=2, pcscaling=1)
# eof_frac=solver.varianceFraction()
# eof_frac

In [ ]:
# eof1[0].plot()

In [ ]:
# # Plot the leading EOF expressed as correlation in the EAC domain.
# import matplotlib.ticker as mticker

# coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
#                             facecolor='none', name='coastline')

# clevs = np.linspace(-.1, .1, 11)

# fig = plt.figure(figsize=(14,10))
# #ax = plt.subplot(1,3,1,projection=ccrs.PlateCarree())
# grid = plt.GridSpec(2, 3, wspace=0.1, hspace=0.7)
# ax = plt.subplot(grid[0, :1],projection=ccrs.PlateCarree())
# feature = ax.add_feature(coast, edgecolor='black',facecolor='gray')
# fill = eof1[0].plot.contourf('lon_rho', 'lat_rho',ax=ax, cmap=plt.cm.RdBu_r,
#                              add_colorbar=False,
#                              transform=ccrs.PlateCarree())
# ax.set_extent([149, 160, -40, -28])
# gl = ax.gridlines(draw_labels=True,
#              color='black', alpha=0.2, linestyle='--')
# gl.xlocator = mticker.FixedLocator([149,150,152,154,156])
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.ylabels_right = False
# gl.xlabels_top = False

# #ax = plt.subplot(1,3,2,projection=ccrs.PlateCarree())
# ax = plt.subplot(grid[0, 1:2],projection=ccrs.PlateCarree())
# feature = ax.add_feature(coast, edgecolor='black',facecolor='gray')
# fill = eof1[1].plot.contourf('lon_rho', 'lat_rho',ax=ax, cmap=plt.cm.RdBu_r,
#                              add_colorbar=False,
#                              transform=ccrs.PlateCarree())
# ax.set_extent([149, 160, -40, -28])
# gl = ax.gridlines(draw_labels=True,
#              color='black', alpha=0.2, linestyle='--')
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.xlocator = mticker.FixedLocator([149,150,152,154,156])
# gl.ylabels_right = False
# gl.xlabels_top = False

# #ax = plt.subplot(1,3,3,projection=ccrs.PlateCarree())
# ax = plt.subplot(grid[0, 2:3],projection=ccrs.PlateCarree())

# feature = ax.add_feature(coast, edgecolor='black',facecolor='gray')
# fill = eof1[2].plot.contourf('lon_rho', 'lat_rho',ax=ax, cmap=plt.cm.RdBu_r,
#                              add_colorbar=False,
#                              transform=ccrs.PlateCarree())
# ax.set_extent([149, 160, -40, -28])
# gl = ax.gridlines(draw_labels=True,
#              color='black', alpha=0.2, linestyle='--')
# gl.xlocator = mticker.FixedLocator([149,150,152,154,156])
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# gl.ylabels_right = False
# gl.xlabels_top = False

# cbar_ax = fig.add_axes([0.21, .485, .6, .02]) #left, bottom, width, height
# fig.colorbar(fill, cax=cbar_ax, orientation="horizontal",extend='both')
# plt.xlabel('Sea surface height [m]')

# # Plot the leading PC time series.
# #plt.figure(figsize=(14,3))
# ax = plt.subplot(grid[1,:])
# pc1[:, 0].plot(color='royalblue', linewidth=2)
# #ax = plt.gca()
# ax.axhline(0, color='k')
# ax.set_ylim(-3, 3)
# ax.set_xlabel('Year')
# ax.set_ylabel('Normalized Units')
# ax.set_title('PC1 Time Series', fontsize=16)
# plt.grid(True)
# plt.savefig('ROMS_SSH_EOF1_3.png', dpi=300,bbox_inches='tight')
# plt.show()

## Calculate and extract transport across isobath

In [ ]:
#calculate transport
U_x200 = ((ds.ubar.xroms.to_grid(hcoord='rho')).where(ds.h<200, drop=True))*ds.dy*ds.h
U_x200 = U_x200.coarsen(eta_rho=23,boundary='trim').mean()

In [ ]:
U_x200.cf.isel(T=0).plot()

In [ ]:
# fill the NaNs with the last value to make it easy to grab the 200m value with a single index
U_x200 = (U_x200.ffill(dim='xi_rho')).isel(xi_rho=102)

In [ ]:
U_x200.drop_vars(['xi_rho','lon_rho','eta_rho']).load()

In [ ]:
df = U_x200.to_dataframe(name='Transport')
df['lat_rho'] = df['lat_rho'].round(decimals=1) # round off the decimals of lat_rho to make it easier to read

In [ ]:
fig = plt.figure(figsize=(12,7))

#set coastline
coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
                            facecolor='none', name='coastline')

#plot
ax = plt.subplot(1,2,1,projection=ccrs.PlateCarree())
feature = ax.add_feature(coast, edgecolor='black',facecolor='gray')
ax.set_extent([148,156,-27,-39])
ds.temp[:,-1,:,:].cf.mean(dim='time').plot.contourf('lon_rho', 'lat_rho',levels=12,cbar_kwargs={'label': 'SST [ $^o$C ]'},
                                        extend = 'both',cmap=cmocean.cm.thermal)

#make pretty gridlines and labels
gl = ax.gridlines(draw_labels=True,
             color='black', alpha=0.2, linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylabels_right = False
gl.xlabels_top = False

#add geostrophic current arrows
# X,Y = np.meshgrid(ds.lon_rho,ds.lat_rho)
# u = ds.u.cf.mean(dim='time')
# v = ds.v.cf.mean(dim='time')
# q = plt.quiver(X,Y,u,v,angles='xy',scale=15,color='k',transform=ccrs.PlateCarree())

plt.subplot(1,2,2)
# ax = sns.violinplot(x="Transport", y="lat_rho", orient ='h',
#                     data=df,scale="width", palette="Set3")
# ax.invert_yaxis()

ax = sns.barplot(x="Transport", y="lat_rho", orient ='h',
                    data=df, palette="Set3")
ax.invert_yaxis()

plt.savefig('Barra_crossshelf_200_transport_bar.png',dpi=300)

In [ ]:
ax = sns.barplot(x="Transport", y="lat_rho", orient ='h',
                    data=df, palette="Set3")
ax.invert_yaxis()

In [ ]:
#calculate transport
U_x200 = ((ds.ubar.xroms.to_grid(hcoord='rho')).where(ds.h<1000, drop=True))*ds.dy*ds.h
U_x200 = U_x200.coarsen(eta_rho=23,boundary='trim').mean()

In [ ]:
# fill the NaNs with the last value to make it easy to grab the 200m value with a single index
U_x200 = (U_x200.ffill(dim='xi_rho')).isel(xi_rho=102)

In [ ]:
U_x200.drop_vars(['xi_rho','lon_rho','eta_rho']).load()

In [ ]:
df = U_x200.to_dataframe(name='Transport')
df['lat_rho'] = df['lat_rho'].round(decimals=1) # round off the decimals of lat_rho to make it easier to read

In [ ]:
fig = plt.figure(figsize=(12,7))

#set coastline
coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
                            facecolor='none', name='coastline')

#plot
ax = plt.subplot(1,2,1,projection=ccrs.PlateCarree())
feature = ax.add_feature(coast, edgecolor='black',facecolor='gray')
ax.set_extent([148,156,-27,-39])
ds.temp[:,-1,:,:].cf.mean(dim='time').plot.contourf('lon_rho', 'lat_rho',levels=12,cbar_kwargs={'label': 'SST [ $^o$C ]'},
                                        extend = 'both',cmap=cmocean.cm.thermal)

#make pretty gridlines and labels
gl = ax.gridlines(draw_labels=True,
             color='black', alpha=0.2, linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylabels_right = False
gl.xlabels_top = False

#add geostrophic current arrows
# X,Y = np.meshgrid(ds.lon_rho,ds.lat_rho)
# u = ds.u.cf.mean(dim='time')
# v = ds.v.cf.mean(dim='time')
# q = plt.quiver(X,Y,u,v,angles='xy',scale=15,color='k',transform=ccrs.PlateCarree())

plt.subplot(1,2,2)
# ax = sns.violinplot(x="Transport", y="lat_rho", orient ='h',
#                     data=df,scale="width", palette="Set3")
# ax.invert_yaxis()

ax = sns.barplot(x="Transport", y="lat_rho", orient ='h',
                    data=df, palette="Set3")
ax.invert_yaxis()

plt.savefig('Barra_crossshelf_1000_transport_bar.png',dpi=300)